In [1]:
from kucoin.client import Trade, Market, User
import pandas as pd

from uuid import uuid4

class KucoinAPI2:
    def __init__(self, sandbox:bool=False) -> None:
        self._credentials ={
            "apiKey":"630e0d3eaa5daa0001fc715c",
            "secret":"17af4d09-29d2-4215-b902-84c59cebfd65",
            "password":"ndaQkccdmkHYANQ6j6@8yKMDP37cSF"
        }
        self._trade_client = Trade(key=self._credentials['apiKey'],
                       secret=self._credentials['secret'],
                       passphrase=self._credentials['password'],
                       is_sandbox=sandbox)
        self._user_client = User(key=self._credentials['apiKey'],
                       secret=self._credentials['secret'],
                       passphrase=self._credentials['password'],
                       is_sandbox=sandbox)
        self._market_client = Market(key=self._credentials['apiKey'],
                       secret=self._credentials['secret'],
                       passphrase=self._credentials['password'],
                       is_sandbox=sandbox)
  
    def authentication_required(fn):
        """Annotation for methods that require auth."""
        def wrapped(self, *args, **kwargs):
            if not self._credentials or "apiKey" not in self._credentials or "secret" not in self._credentials or "password" not in self._credentials: 
                raise Exception(f"You must be authenticated to use this method {fn}") 
            else:
                return fn(self, *args, **kwargs)
        return wrapped
    
    @authentication_required
    def getUSDTBalance(self)->float:
        try:
            return self.__getBalance('USDT','trade')
        except Exception as err:
            raise err
        
    @authentication_required
    def __getBalance(self,currency:str,account_type:str='trade')->float:
        try:
            return float(pd.DataFrame(self._user_client.get_account_list()).query('currency=="{}" and type=="{}"'.format(currency,account_type)).available)
        except TypeError:
            raise Exception(f"You don't have {currency} in your {account_type} account.")
        except Exception as err:
            raise err
        
    def getCurrentPrice(self, symbol:str)->float:
        try:
            return float(self._market_client.get_ticker(symbol=symbol)['price'])
        except Exception as err:
            raise err

    def getAmountForTrade(self, symbol:str,pct_wallet:int=1)->str:
        quote_min_size = float(pd.DataFrame(self._market_client.get_symbol_list()).query('symbol=="{}"'.format(symbol)).quoteMinSize)
        precision = self._market_client.get_currency_detail(symbol.split('-')[0])['precision']
        if symbol.split('-')[1] == 'USDT':
            usdt_amount = self.getUSDTBalance()*pct_wallet
            if usdt_amount < quote_min_size:
                raise Exception(f"USDT amount is less than the minimum amount for this trade {symbol}")
            else:
                raw_string_price =  str(usdt_amount/self.getCurrentPrice(symbol))
                return  f"{raw_string_price.split('.')[0]}.{raw_string_price.split('.')[1][:precision]}"
        else:
            quote_amount = self.__getBalance(symbol.split('-')[1])*pct_wallet
            if quote_amount < quote_min_size:
                raise Exception(f"{symbol.split('-')[1]} amount is less than the minimum amount for this trade {symbol}")
            else:
                raw_string_price =  str(quote_amount/self.getCurrentPrice(symbol))
                return  f"{raw_string_price.split('.')[0]}.{raw_string_price.split('.')[1][:precision]}"
        
    @authentication_required
    def placeMarketOrder(self,symbol:str,side:str,amount:str)-> dict:
        assert side.lower() in ["buy","sell"]
        
        result = self._trade_client.create_market_order(f'{symbol}',side=side.lower(),clientOid=str(uuid4()),size=amount)
        self.last_orderId = result['orderId']

        trade_details = self._trade_client.get_order_details(self._last_orderId)
        self.last_side = side.lower()
        self.qty_in_position=trade_details['dealSize']
        self.quote_qty_in_position=trade_details['dealFunds']
        
        return result
         
api_ = KucoinAPI2()    
amount = api_.getAmountForTrade('BTC-USDT',0.1)
print(amount)
    
    #@authentication_required    
    #def placeMarketOrder(self,symbol:str,side:str,amount:str)->dict:
    #    assert side.lower() in ["buy","sell"]
    #    self._session.create_market_order(f'{symbol}/USDT', side.lower(), amount)
    #
    #@authentication_required   
    #def cancelOrder(self, orderId:str)->dict:
    #    return self._session.cancel_order(orderId)
    #
    #    
    #def getAmountForSellingAll(self,symbol:str)->str:
    #    return self._session.amount_to_precision(f'{symbol}/USDT', self.qty_in_position)
    #



0.00015724


In [110]:
import time


print(api_.placeMarketOrder('BTC-USDT','buy',amount))
time.sleep(5)
print(api_.placeMarketOrder('BTC-USDT','sell',amount))

{'orderId': '632cba2d3b7e10000163e59f'}
{'orderId': '632cba3211dc1a0001c5531c'}


In [2]:
api_._trade_client.get_order_details('632cba2d3b7e10000163e59f')

{'id': '632cba2d3b7e10000163e59f',
 'symbol': 'BTC-USDT',
 'opType': 'DEAL',
 'type': 'market',
 'side': 'buy',
 'price': '0',
 'size': '0.00015634',
 'funds': '0',
 'dealFunds': '3.023897012',
 'dealSize': '0.00015634',
 'fee': '0.003023897012',
 'feeCurrency': 'USDT',
 'stp': '',
 'stop': '',
 'stopTriggered': False,
 'stopPrice': '0',
 'timeInForce': 'GTC',
 'postOnly': False,
 'hidden': False,
 'iceberg': False,
 'visibleSize': '0',
 'cancelAfter': 0,
 'channel': 'API',
 'clientOid': 'e6365a9b-66eb-4ca3-a177-4ea32aaab417',
 'remark': None,
 'tags': None,
 'isActive': False,
 'cancelExist': False,
 'createdAt': 1663875629014,
 'tradeType': 'TRADE'}

In [4]:
api_.getAmountForTrade('BTC-USDT',0.1)

'0.00015721'

In [3]:
api_.getUSDTBalance()*0.1

3.022157624

In [1]:
from datetime import datetime
import ccxt
import json

class KucoinAPI:
    def __init__(self) -> None:
        #self._credentials = {
        #    "apiKey":"62fa6d4f37a609000198c9ca",
        #    "secret":"7ab549c3-5fb9-46f6-8f5a-4ba63556102d",
        #    "password":"oHCsD5dDK&HrskYC8a9cYp9N@hgRRg9"
        #}#sandbox
    #    self._credentials = {
    #"apiKey":"630e0d3eaa5daa0001fc715c",
    #"secret":"17af4d09-29d2-4215-b902-84c59cebfd65",
    #"password":"ndaQkccdmkHYANQ6j6@8yKMDP37cSF"
    #    }#sub account snipping
        self._credentials ={
            "apiKey":"62a05a9ccf9fd60001e2ff9e",
            "secret":"578687f6-5420-46d7-9b7a-26a737713cfb",
            "password":"3FiF$4sR6p5Y9fPnSNijL?HaGcd@A7"
        }
        self._session = ccxt.kucoin(self._credentials)
        #self._session.set_sandbox_mode(True)

    
    def authentication_required(fn):
        """Annotation for methods that require auth."""
        def wrapped(self, *args, **kwargs):
            if not self._credentials or "apiKey" not in self._credentials or "secret" not in self._credentials or "password" not in self._credentials: 
                raise Exception(f"You must be authenticated to use this method {fn}") 
                #sys.exit()
            else:
                return fn(self, *args, **kwargs)
        return wrapped
        
    @authentication_required    
    def placeMarketOrder(self,symbol:str,side:str,amount:str)->dict:
        assert side.lower() in ["buy","sell"]
        self._session.create_market_order(f'{symbol}/USDT', side.lower(), amount)

    
    @authentication_required     
    def placeLimitOrder(self):
        pass  
    
    @authentication_required   
    def cancelOrder(self, orderId:str)->dict:
        return self._session.cancel_order(orderId)
    
    @authentication_required
    def getUSDTBalance(self)->float:
        try:
            for coin in self._session.fetchBalance()['info']['data']:
                if coin['currency']=='USDT' :
                    return float(coin['balance'])
        except Exception as err:
            raise err
        
    def getCurrentPrice(self,symbol:str)->float:
        try:
            return self._session.fetch_ticker(f'{symbol}/USDT')["ask"]
        except Exception as err:
            raise err
        
    def getAmountForSellingAll(self,symbol:str)->str:
        return self._session.amount_to_precision(f'{symbol}/USDT', self.qty_in_position)
    
    def getAmountForTrade(self, symbol:str,pct_wallet:int=1)->str:
        if self._session.load_markets()[f'{symbol}/USDT']['limits']['amount']['min']<self.getUSDTBalance()*pct_wallet/self.getCurrentPrice(f'{symbol}'):
            print(self.getUSDTBalance()*pct_wallet/self.getCurrentPrice(f'{symbol}'))
            return self._session.amount_to_precision(f'{symbol}/USDT', self.getUSDTBalance()*pct_wallet/self.getCurrentPrice(f'{symbol}'))
        else:
            raise Exception(f"Not enough USDT to trade {symbol}") 
        
    def convert_amount_to_precision(self, symbol, amount):
        return self._session.amount_to_precision(symbol, amount)
        


In [77]:
kucoin_client = KucoinAPI()

In [78]:
kucoin_client.getCurrentPrice('BTC')

19522.7

In [102]:
kucoin_client.getUSDTBalance()

30.22679626

In [103]:
kucoin_client.getAmountForTrade('BTC',1)

0.0015723550507441258


'0.00157235'